In [7]:
from google.colab import drive
drive.mount('/content/drive/')
!ls '/content/drive/MyDrive/07MIAR_PracticaObservacional/code/'
import sys
sys.path.append('/content/drive/MyDrive/07MIAR_PracticaObservacional/code/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
convolutional_autoencoder.py  __pycache__	  utils.py
my_data_generator.py	      training_CAE.ipynb


In [10]:
# Imports necesarios
import tensorflow as tf
import pandas as pd
from my_data_generator import DataGenerator
from convolutional_autoencoder import ConvAutoencoder
from utils import save_reconstructed_images, create_environment, create_json
from tensorflow.keras.models import load_model
import os
import numpy as np
import cv2

In [11]:
#Hyper-parameters
input_dim = (128, 128, 1)
encoder_conv_filters = [32, 64, 64, 64]
encoder_conv_kernel_size = [3, 3, 3, 3]
encoder_conv_strides = [2, 2, 2, 2]
decoder_conv_t_filters = [64, 64, 32, 1]
decoder_conv_t_kernel_size = [3, 3, 3, 3]
decoder_conv_t_strides = [2, 2, 2, 2]
sess = tf.function()
z_dim = 200
lr = 0.0005
batch_size = 8
epochs = 50
r_loss_factor = 0.4
is_training = True

In [15]:
base_folder = "/content/drive/MyDrive/07MIAR_PracticaObservacional/"
#I/O paths
run_folders = {"tsv_path": base_folder + "data/partition.csv", 
               "data_path": base_folder + "data/images/", 
               "model_path": base_folder + "data/Models/", 
               "results_path": base_folder + "data/Results/", 
               "log_filename": base_folder + "data/Results/log/CAE.log"
               }

In [16]:
# Creando el entramado de directorios
create_environment(run_folders)

In [17]:
hyperparameters = {"input_dim": input_dim, 
                   "encoder_conv_filters": encoder_conv_filters, 
                   "encoder_conv_kernel_size":encoder_conv_kernel_size, 
                   "encoder_conv_strides":encoder_conv_strides, 
                   "decoder_conv_t_filters":decoder_conv_t_filters, 
                   "decoder_conv_t_kernel_size":decoder_conv_t_kernel_size, 
                   "decoder_conv_t_strides":decoder_conv_t_strides, 
                   "z_dim":z_dim, 
                   "lr":lr, 
                   "batch_size":batch_size, 
                   "epochs":epochs, 
                   "r_loss_factor":r_loss_factor, 
                   "opt":"Adam", 
                   "loss_function":"mse",
                   "data_path": run_folders["data_path"]
                   }
create_json(hyperparameters, run_folders)

In [18]:
# Fase de entrenamiento
if is_training:
  # Carga de datos
  df_pneumo_2d = pd.read_csv(run_folders["tsv_path"], sep=";")
  df_pneumo_2d.columns = ['ImageID', 'Partition', 'Class']
  data_filter = df_pneumo_2d['Partition']=='train'
  # Cargador de datos training
  data_flow_train = DataGenerator(df_pneumo_2d[data_filter], 
                                  input_dim[1], 
                                  input_dim[0], 
                                  input_dim[2], 
                                  batch_size=batch_size, 
                                  path_to_img=run_folders["data_path"],
                                  shuffle = True)
  # Cargador de datos validation
  data_filter = df_pneumo_2d['Partition']=='val'
  data_flow_val = DataGenerator(df_pneumo_2d[data_filter], 
                                  input_dim[1], 
                                  input_dim[0], 
                                  input_dim[2], 
                                  batch_size=batch_size, 
                                  path_to_img=run_folders["data_path"],
                                  )
  # Creando el CAE
  my_CAE = ConvAutoencoder(input_dim, 
                           encoder_conv_filters, 
                           encoder_conv_kernel_size, 
                           encoder_conv_strides, 
                           decoder_conv_t_filters,
                           decoder_conv_t_kernel_size, 
                           decoder_conv_t_strides,
                           z_dim)
  
  # Construyendo arquitectura
  my_CAE.build(use_batch_norm=True, use_dropout=False, VCAE=False)
  print(my_CAE.model.summary())

  # Compilando el CAE
  my_CAE.compile(learning_rate=lr, r_loss_factor=r_loss_factor)

  # Entrenando el CAE
  steps_per_epoch = len(data_flow_train)
  H = my_CAE.train(data_flow_train, epochs, steps_per_epoch, data_flow_val, run_folders)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 128, 128, 1)]     0         
_________________________________________________________________
encoder_conv0 (Conv2D)       (None, 64, 64, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 32)        0         
_________________________________________________________________
encoder_conv1 (Conv2D)       (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
44/44 [==============================] - 398s 9s/step - loss: 0.0441 - val_loss: 0.0518

Epoch 00001: saving model to /content/drive/MyDrive/07MIAR_PracticaObservacional/data/Models/exp_0001/weights/CAE_weights.h5
Epoch 2/50
44/44 [==============================] - 19s 423ms/step - loss: 0.0194 - val_loss: 0.0391

Epoch 00002: saving model to /content/drive/MyDrive/07MIAR_PracticaObservacional/data/Models/exp_0001/weights/CAE_weights.h5
Epoch 3/50
44/44 [==============================] - 19s 422ms/step - loss: 0.0140 - val_loss: 0.0395

Epoch 00003: saving model to /content/drive/MyDrive/07MIAR_PracticaObservacional/data/Models/exp_0001/weights/CAE_weights.h5
Epoch 4/50
44/44 [==============================] - 19s 424ms/step - loss: 0.0118 - val_loss: 0.0374

Epoch 00004: saving model to /content/drive/MyDrive/07MIAR_PracticaObservacional/data/Models/exp_0001/weights/CAE_weights.h5
Epoch 5/50
44/44 [==============================] - 19s 423ms/step - loss: 0.0095 - val_loss: 

In [20]:
is_training = True
# Carga de datos
df_pneumo_2d = pd.read_csv(run_folders["tsv_path"], sep=";")
df_pneumo_2d.columns = ['ImageID', 'Partition', 'Class']
data_filter = df_pneumo_2d['Partition']=='test'
# Cargador de datos training
data_flow_test = DataGenerator(df_pneumo_2d[data_filter], 
                                input_dim[1], 
                                input_dim[0], 
                                input_dim[2], 
                                batch_size=batch_size, 
                                path_to_img=run_folders["data_path"],
                                shuffle = False)

if not is_training:
  my_CAE = ConvAutoencoder.load_model(run_folders)

print(my_CAE.model.summary())

example_batch = data_flow_test.__getitem__(index=0)
example_images = example_batch[0]

y_pred = my_CAE.model.predict(example_images)
print(y_pred.shape)

save_reconstructed_images(example_images, y_pred, run_folders)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 128, 128, 1)]     0         
_________________________________________________________________
encoder_conv0 (Conv2D)       (None, 64, 64, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 32)        0         
_________________________________________________________________
encoder_conv1 (Conv2D)       (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0   